<center><h2><strong><font color="blue"> Advanced Programming for Data Science (APDS)</font></strong></h2></center>

<center><img alt="" src="images/covers/taudata-cover.jpg" style="height: 200px;" /></center>

<center><h2><strong><font color="blue">APDS-07: Introduction to Cython: The Compiled Python</font></strong></h2></center>

<b><center><h3>(C) Taufik Sutanto</h3></center>
* .

# <center><font color="blue"> Outline </font></center>

* Previous (Prime) Exercise + JIT
* Dynamic Typing and Compiler
* Introduction to Cython: Compiled Python
* 

In [ ]:
!pip install numpy pandas numba seaborn cython  --q

In [18]:
# Importing Some Python Modules
import warnings; warnings.simplefilter('ignore')
import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt
import numba, cython, time
from numba import jit

plt.style.use('bmh'); sns.set()
np.random.seed(420)

# <center><font color="red"> Previous Exercise </font></center>

<center><img alt="" src="images/exercise-question.jpg" style="height: 250px;" /></center>

> The Prime

<center><img alt="" src="images/prime-definition.jpg" style="height: 250px;" /></center>

# <center><font color="red"> Prime Numbers </font></center>

* integers ≥ 2
* "2" is the only even prime
* Have only 2 factors: "1" and "itself"
* That means it is not **divisible** by any number starting from 2 upto n-1.

In [15]:
def isPrime(n):
    if n <= 2 or not(isinstance(n, int)):
        return False
    elif n==2:
        return True
    else:
        for i in range(2, n-1):
            if n%i==0:
                return False
        return True

In [18]:
test_data = [3, 72, 131, 137, 140, 145, 151]
for number in test_data:
    print(isPrime(number), end=", ")

True, False, True, True, False, False, True, 

## Make it More Efficient?

* Remember All even number except 2 are all not a prime number.
* First you need to remember how "range" work in Python

In [11]:
list(range(2, 10))

[2, 3, 4, 5, 6, 7, 8, 9]

### therefore it is more efficient if we just check whether a number is divisible or not to "i" by using "step":

In [13]:
list(range(3, 10, 2))

[3, 5, 7, 9]

### Hence the Function now becomes:

In [14]:
def isPrime(n):
    if n <= 2 or not(isinstance(n, int)):
        return False
    elif n==2:
        return True
    elif n%2==0: # Even numbers are not prime (except 2)
        return False
    else:
        for i in range(3, n-1, 2):
            if n%i==0:
                return False
        return True

In [20]:
test_data = [3, 72, 131, 137, 140, 145, 151]
for number in test_data:
    print(isPrime(number), end=", ")

True, False, True, True, False, False, True, 

# Bisa lebih efisien lagi? (terutama jika bilangannya besar)

<center><img alt="" src="images/cython-meme.jpg" style="height: 400px;"/></center>

In [30]:
m = np.ceil(np.sqrt(99))
m, type(m), int(m)

(np.float64(10.0), numpy.float64, 10)

In [10]:
def isPrime(n):
    if n <= 2 or not(isinstance(n, int)):
        return False
    elif n==2:
        return True
    elif n%2==0: # Even numbers are not prime (except 2)
        return False
    else:
        m = np.ceil(np.sqrt(n)) + 1 
        for i in range(3, int(m), 2):
            if n%i==0:
                return False
        return True

In [4]:
test_data = [3, 72, 131, 137, 140, 145, 151]
for number in test_data:
    print(isPrime(number), end=", ")

True, False, True, True, False, False, True, 

# Take Away:

1. Make the code efficient using the programming language knowledge (i.e. syntax)
2. Make the code more efficient using logic
3. Make the code even more efficient using domain knowledge
4. Try JIT/Compiled Code to make it faster
5. Try Parallel Processing to optimize it to our machine (hardware)

## The Exercise is not done yet!

* Generate first n prime numbers
      - input n (integer)
      - Do a (while) loop to produce prime numbers one-by-one. 
* Try JIT

In [6]:
def myPrime(n):
    results = []
    number = 2 #Start from the lowest Prime number
    while len(results)<n:
        if isPrime(number):
            results.append(number)
        number += 1 
    return results

In [11]:
myPrime(20)

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73]

In [17]:
N = 10**5 #Testing with generating 100,000 prime numbers
start = time.time()
dPrimes = myPrime(N)
end = time.time()
print("Elapsed Time = {:.6f} seconds".format(end-start))

Elapsed Time = 3.845077 seconds


# Let's Try JIT

In [22]:
@jit(nopython=True)
def isPrimeJIT(n):
    if n <= 2 or not(isinstance(n, int)):
        return False
    elif n==2:
        return True
    elif n%2==0: # Even numbers are not prime (except 2)
        return False
    else:
        m = np.ceil(np.sqrt(n))+1
        for i in range(3, int(m), 2):
            if n%i==0:
                return False
        return True

isPrimeJIT(5) # initialize the JIT function

True

In [23]:
@jit(nopython=True)
def myPrimeJIT(n):
    results = []
    number = 2 #Start from the lowest Prime number
    while len(results)<n:
        if isPrimeJIT(number):
            results.append(number)
        number += 1 
    return results

myPrimeJIT(3) # initialize the JIT function

[3, 5, 7]

In [25]:
N = 10**5 #Testing with generating 100,000 prime numbers
start = time.time()
dPrimes = myPrimeJIT(N)
end = time.time()
print("Elapsed Time = {:.6f} seconds".format(end-start))

Elapsed Time = 0.077909 seconds


In [26]:
# Validate the Results (always do this)
dPrimes[:20]

[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73]

In [28]:
# The improvements ... ~ 50 times faster
3.845077/0.077909

49.35343798534187

## Exercise by using Numba's "Explicit Type declaration" and "Parallelization"

# <center><font color="blue"> Introduction to Cython: The Compiled Python </font></center>

<center><img alt="" src="images/APDS/apds-07-illustrations-cython.jpg" style="height: 350px;" /></center>

Cython is a powerful superset of the Python programming language designed to give Python code near-C performance by allowing developers to add optional static type declarations and compile Python-like syntax into optimized C extensions. It bridges the gap between Python’s ease of use and C’s execution speed, making it especially valuable for numerical computing, data processing, machine learning, and performance-critical sections of large applications. By integrating seamlessly with existing Python code and libraries, Cython enables developers to accelerate bottlenecks without rewriting entire modules in C, while still maintaining Python’s readability and flexibility.

* https://cython.org/
* https://cython.readthedocs.io/
* https://pypi.org/project/Cython/

## The "Hello World"

# Ref:

* https://towardsdatascience.com/run-your-python-code-up-to-80x-faster-using-the-cython-library/
* https://medium.com/@migueloteropedrido/cythonizing-your-code-for-beginners-6c7eba2b38ae
* https://www.geeksforgeeks.org/python/optimizing-python-code-with-cython/
* https://cython.readthedocs.io/en/latest/src/tutorial/index.html

<center><h2><strong><font color="blue">End of Module</font></strong></h2></center>
<hr>
<center><img alt="" src="images/meme-cartoon/cython-meme.jpg" style="height: 400px;"/></center>